# Tracking Objects with YOLO

In [ ]:
from ultralytics import YOLO
import os

os.chdir("..")

## Data Preparation

Our data is the RBK football dataset. It contains images of the ball, football players and referees with bounding box annotations, class labels and other metadata.

There are several games recorded, each in its own folder. Where each folder has data.yml file describing the dataset structure, a train.txt file listing all training images, and subfolders for images and labels. The structure is as follows:

```
.data/
└── Football2025
    ├── RBK-AALESUND
    │   ├── data
    │   │   └── images
    │   │       └── train
    │   └── labels
    │       └── train
    ├── RBK-BODO
    │   ├── part1
    │   │   └── RBK_BODO_PART1
    │   │       ├── data
    │   │       │   └── images
    │   │       │       └── train
    │   │       └── labels
    │   │           └── train
    │   ├── part2
    │   │   └── RBK_BODO_PART2
    │   │       ├── data
    │   │       │   └── images
    │   │       │       └── train
    │   │       └── labels
    │   │           └── train
    │   └── part3
    │       └── RBK_BODO_PART3
    │           ├── data
    │           │   └── images
    │           │       └── train
    │           └── labels
    │               └── train
    ├── RBK-FREDRIKSTAD
    │   ├── data
    │   │   └── images
    │   │       └── train
    │   └── labels
    │       └── train
    ├── RBK-HamKam
    │   ├── data
    │   │   └── images
    │   │       └── train
    │   └── labels
    │       └── train
    └── RBK-VIKING
        ├── data
        │   └── images
        │       └── train
        └── labels
            └── train
```

RBK_BODØ has three parts and is the only dataset that annotated the correct tracks for the players, referee and the ball. The other datasets only have bounding box annotations for each frame.

To prepare the data for training, we will merge all the datasets into a single folder structure that YOLOv8 can work with. We will also create a single train.txt file that lists all the training images from all datasets.

## Load single dataset

In [ ]:
# Configure the tracking parameters and run the tracker
model = YOLO("yolo11l.pt")


## Tracking with YOLO

In [ ]:
folder_path = "data/Football2025/RBK-BODO/part2/RBK_BODO_PART2/data/images/train"

track_results = []

for frame_number, img_file in enumerate(os.listdir(folder_path)):
    if img_file.endswith(".jpg") or img_file.endswith(".png"):
        img_path = os.path.join(folder_path, img_file)
        results = model.track(source=img_path, show=False, persist=True, tracker="bytetrack.yaml")
        track_results.append(results[0])
    if frame_number >= 10:  # Limit to first 10 frames for testing
        break    



In [ ]:
track_results[7].boxes.cls


# Track images


In [ ]:
import pandas as pd
import random

df = pd.DataFrame(columns=["frame", "id", "x", "y", "w", "h", "conf", "class", "visibility"])
for frame_number, result in enumerate(track_results, start=1):
    
    xywh = result.boxes.xywh  # Bounding box coordinates in (x_center, y_center, width, height) format
    ids = result.boxes.id  # Object IDs assigned by the tracker
    if ids is None:
        ids = random.sample(range(1000, 9999), len(xywh))  # Assign random IDs if None

    classes = result.boxes.cls  # Class IDs of detected objects
    conf = result.boxes.conf  # Confidence scores of detections

    for box, obj_id, cls_id, confidence in zip(xywh, ids, classes, conf):

        df = pd.concat([df, pd.DataFrame({
            "frame": frame_number,
            "id": int(obj_id),
            "x": float(box[0]),
            "y": float(box[1]),
            "w": float(box[2]),
            "h": float(box[3]),
            "conf": float(confidence),
            "class": int(cls_id),
            "visibility": -1.0
        }, index=[0])], ignore_index=True)


df.to_csv("pred.txt", index=False, header=False)